In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import TensorDataset, DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append("../src/")
import importlib
import structures
importlib.reload(structures)
from structures import *
import losses
importlib.reload(losses)
from losses import *
import trainer
importlib.reload(trainer)
from trainer import *
#dtype = torch.float32
dtype = torch.double
torch.set_default_dtype(dtype)
torch.autograd.set_detect_anomaly(True)
# Set the device to CUDA if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

import deepspeed
import random
sys.path.append("/mnt/home/yjo10/ceph/myutils")
import plt_utils as pu

Using device: cuda
[2025-04-13 20:04:07,603] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/mnt/sw/nix/store/r3bwp9b2501bv77y6g1nwkb483p0y9z2-cuda-12.3.2/lib64/libcufile.so: undefined reference to `dlvsym'
/mnt/sw/nix/store/r3bwp9b2501bv77y6g1nwkb483p0y9z2-cuda-12.3.2/lib64/libcufile.so: undefined reference to `dlopen'
/mnt/sw/nix/store/r3bwp9b2501bv77y6g1nwkb483p0y9z2-cuda-12.3.2/lib64/libcufile.so: undefined reference to `dlclose'
/mnt/sw/nix/store/r3bwp9b2501bv77y6g1nwkb483p0y9z2-cuda-12.3.2/lib64/libcufile.so: undefined reference to `dlerror'
/mnt/sw/nix/store/r3bwp9b2501bv77y6g1nwkb483p0y9z2-cuda-12.3.2/lib64/libcufile.so: undefined reference to `dlsym'
/mnt/sw/nix/store/r3bwp9b2501bv77y6g1nwkb483p0y9z2-cuda-12.3.2/lib64/libcufile.so: undefined reference to `shm_open'
/mnt/sw/nix/store/r3bwp9b2501bv77y6g1nwkb483p0y9z2-cuda-12.3.2/lib64/libcufile.so: undefined reference to `shm_unlink'
collect2: error: ld returned 1 exit status


## First Attempt

### data load

In [5]:
def normalizer(_data):
    _max = torch.abs(_data.max())
    data_max = torch.ones(_data.size(), device=device)*_max
    _data = _data/data_max
    return _data, _max
    
def denormalizer(_data, _max):
    data_max = torch.ones(_data.size(), device=device)*_max
    return _data*data_max

# Example unsupervised data: 100 samples with 10 features each
data = np.load("../data/two_body_train_data_new.npy")
num_samples = data.shape[0]
num_samples_start = int(data.shape[0]*0.8)
num_samples_start = 0
num_samples_end   = data.shape[0]
num_samples = num_samples_end - num_samples_start
num_features = data.shape[1]

# Placeholder for input (magnitudes of velocities and accelerations)
data_org = torch.tensor(data[num_samples_start:num_samples_end,:], dtype=dtype).to(device)
data = torch.empty((num_samples,6+num_features), dtype=dtype).to(device)
data[:,6:] = data_org


# Magnitudes vel, acc, mass
temp, tmax = normalizer(data_org[:,4:7])
vel = denormalizer(torch.norm(temp, p=2, dim=1),tmax)
acc = torch.empty((num_samples,4), dtype=dtype).to(device)

temp,tmax = normalizer(data_org[:,7:10])
acc[:,0] = denormalizer(torch.norm(temp, p=2, dim=1),tmax)
temp,tmax = normalizer(data_org[:,10:13])
acc[:,1] = denormalizer(torch.norm(temp, p=2, dim=1),tmax)
temp,tmax = normalizer(data_org[:,13:16])
acc[:,2] = denormalizer(torch.norm(temp, p=2, dim=1),tmax)
temp,tmax = normalizer(data_org[:,16:19])
acc[:,3] = denormalizer(torch.norm(temp, p=2, dim=1),tmax)
mass = data_org[:,0]

data[:,0] = vel
data[:,1:5] = acc
data[:,5] = mass

# Normalize the data
eps = 1e-5
data_min = data[:,:6].min(axis=0, keepdim=True).values
data_max = data[:,:6].max(axis=0, keepdim=True).values
data[:,:6] = (data[:,:6] - data_min) / (data_max - data_min)+eps


# Wrap the feature tensor in a TensorDataset
# Each item from the dataset will be a tuple containing one tensor (X[i],)
dataset = TensorDataset(data)

# Define the proportion for the test set (e.g., 20%)
test_ratio = 0.2
num_test = int(num_samples * test_ratio)
num_train = num_samples - num_test

# Use random_split to split the dataset into train and test subsets
# Define generator with a fixed seed
generator = torch.Generator().manual_seed(42)
train_dataset, test_dataset = random_split(dataset, [num_train, num_test], generator=generator)

# Create DataLoaders for the training and testing datasets
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print(data.shape)

torch.Size([590, 48])


### Training

In [ ]:
import structures
importlib.reload(structures)
from structures import *
import losses
importlib.reload(losses)
from losses import *
import trainer
importlib.reload(trainer)
from trainer import *

input_mask = np.r_[0:6]
input_size = len(input_mask)
#hidden_dims = [32,64,64,32]     # Number of hidden neurons
#hidden_dims = [8,16,16,4]     # Number of hidden neurons
hidden_dims = [12,12,6]     # Number of hidden neurons
#hidden_dims = [16,128,32,8]     # Number of hidden neurons
#hidden_dims = [8,8]     # Number of hidden neurons
output_size = 2     # Number of output 
TargetEnergyError = 1e-3
weights = {"time_step":10, "energy_loss":1}

# Instantiate the model
try: 
    del mode
    l
    print("model deleted")
except:
    pass


ds_config = {
    "train_batch_size": batch_size,
    "gradient_accumulation_steps": 1,
    "optimizer": {
        "type": "Adam",
        "params": {"lr": 1e-3, "betas": [0.9, 0.999], "eps": 1e-8}
    },
    "fp32": {
        "enabled": True  # Enables mixed precision training
    },
    "zero_optimization": {
        "stage": 1  # Enable ZeRO Stage 1 for memory optimization
    }
}


#model = SimpleNN(input_size, hidden_size, output_size).to(device)
model1 = FullyConnectedNN(input_dim=input_size, output_dim=output_size, hidden_dims=hidden_dims, activation='relu', dropout=0.0, output_positive=False).to(device)
model2 = FullyConnectedNN(input_dim=input_size, output_dim=output_size, hidden_dims=hidden_dims, activation='relu', dropout=0.0, output_positive=False).to(device)
model3 = FullyConnectedNN(input_dim=input_size, output_dim=output_size, hidden_dims=hidden_dims, activation='relu', dropout=0.0, output_positive=False).to(device)


model_engine, optimizer, _, _ = deepspeed.initialize(
    model=model,
    model_parameters=model.parameters(),
    config=ds_config
)

# Define the loss function (CrossEntropyLoss is common for classification tasks)
criterion = CustomizableLoss3DM(nParticle=2, nAttribute=20, nBatch=batch_size,alpha=weights['time_step'], beta=weights['energy_loss'], gamma=weights['energy_loss'], TargetEnergyError=TargetEnergyError,
                            data_min=data_min, data_max=data_max,device=device)

# Define the optimizer (Stochastic Gradient Descent in this example)
#optimizer = optim.SGD(model.parameters(), lr=0.001)
#optimizer = optim.Adam(model.parameters(), lr=1e-5)
#optimizer = optim.AdamW(model.parameters(), lr=0.00001)
#optimizer = optim.AdamW(model.parameters(), lr=0.00001)



# Paths to save best and last models
base_path = "/mnt/home/yjo10/projects/AITimeStepper/data/models/two_body/"
best_val_path = base_path + "best_model_loss.pth"
best_timestep_path = base_path + "best_model_timestep.pth"
#last_model_path = base_path + f"last_model_{epoch}.pth"
best_val_loss = float("inf")
best_largest_timestep = 0
with open(base_path+"c++/normalization_factors.txt", "w") as f:
    f.write(f"{data_min[0][0]} {data_min[0][1]} {data_min[0][2]} {data_min[0][3]} {data_min[0][4]} {data_min[0][5]}\n")    
    f.write(f"{data_max[0][0]} {data_max[0][1]} {data_max[0][2]} {data_max[0][3]} {data_max[0][4]} {data_max[0][5]}\n")    

example_input = torch.randn(1, input_size).to(device)
exp_time_step = 1
exp_energy_loss = 1
# Training routine
num_epochs = 1000
lists = {"training_loss":[], "val_loss":[], "training_energy":[], "val_energy":[], "training_time_step":[], "val_time_step":[]}

for epoch in range(num_epochs):

    #train_loss, train_energy = \
    #    train_one_epoch(model, optimizer, criterion, train_loader, input_mask, weights, device)
    train_res = train_one_epoch_deepspeed(model_engine, optimizer, criterion, train_loader, input_mask, weights, device)
        
    #test_loss, energy_error, energy_error_std, energy_error_fiducial, energy_error_fiducial_std, energy_pred, energy_init, time_step, time_step_fiducial = \
    #    validate(model, criterion, test_loader, input_mask, weights, device)
    val_res = validate_deepspeed(model_engine, criterion, test_loader, input_mask, weights, device)

    lists["training_loss"].append(train_res["train_loss"])
    lists["training_energy"].append(train_res["energy_error"])
    lists["training_time_step"].append(train_res["time_step"])

    lists["val_loss"].append(val_res["val_loss"])
    lists["val_energy"].append(val_res["energy_error"])
    lists["val_time_step"].append(val_res["time_step"])

    if epoch > 5:
        # Save the best model based on validation loss
        if val_res['val_loss'] < best_val_loss:
            best_val_loss = val_res['val_loss']
            torch.save(model.state_dict(), best_val_path)
            #traced_model = torch.jit.trace(model, example_input)
            traced_model = torch.jit.script(model)
            traced_model.save(base_path + f"c++/best_model_loss.pt")
            print(f"--> Best model saved at epoch {epoch} with val loss {best_val_loss:.4e}")
        
        # Save the best model based on validation largest timestep
        if val_res['time_step'] > best_largest_timestep:
            best_largest_timestep = val_res['time_step']
            torch.save(model.state_dict(), best_timestep_path)
            #traced_model = torch.jit.trace(model, example_input)
            traced_model = torch.jit.script(model)
            traced_model.save(base_path + f"c++/best_model_timestep.pt")
            print(f"--> Best model saved at epoch {epoch} with largest timestep {best_largest_timestep:.4e}")

    # Save the last model after every epoch (or just once at the end)
    torch.save(model.state_dict(), base_path + f"model_{epoch}.pth")
    #traced_model = torch.jit.trace(model, example_input)
    traced_model = torch.jit.script(model)
    traced_model.save(base_path + f"c++/model_{epoch}.pt")



    fig, axes = pu.generateAxesForMultiplePlots(shape=(1,3),figsize=(10,10),hspace=0.1,wspace=0.1,
                                    gridspec=None)
    for i, (key, value) in enumerate(lists.items()):
        y = i//2
        axes[y].plot(range(epoch+1), value, label=key)
        axes[y].scatter(range(epoch+1), value)
    axes[0].set_yscale("log")
    axes[2].set_yscale("log")

    axes[0].grid()
    axes[1].grid()
    axes[2].grid()


    axes[2].axhline(y=val_res['time_step_fiducial'], c='r',alpha=0.5,zorder=-1,label="Timestep Fiducial")
    axes[1].axhline(y=np.log10(TargetEnergyError), c='r',alpha=0.5,zorder=-1,label="Target energy")
    #plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0.)
    axes[0].legend()
    axes[1].legend()
    axes[2].legend()
    #plt.tight_layout()
    axes[0].set_ylabel("Loss")
    axes[1].set_ylabel("log(Relative Error of Energy)")
    axes[2].set_ylabel("Timestep")
    axes[2].set_xlabel("epoch")
    plt.savefig("learning_curve.png",dpi=100)
    plt.close()



    fig, axes = pu.generateAxesForMultiplePlots(shape=(1,2),figsize=(10,15),hspace=0.1,wspace=0.1,
                                    gridspec=None)
    # Get one batch from the validation loader
    dataiter = iter(train_loader)
    inputs, = next(dataiter)

    # Determine how many samples to take (if batch size < 50, take the whole batch)
    num_samples = min(50, batch_size)

    # Randomly select indices from the batch
    indices = random.sample(range(batch_size), num_samples)
    sampled_inputs = inputs[:,input_mask][indices]
    sampled_targets = inputs[indices,25]

    # Compute model predictions with no gradient computation
    with torch.no_grad():
        sampled_predictions = model(sampled_inputs)

    # Convert tensors to numpy arrays for plotting
    sampled_predictions = sampled_predictions[:,0].cpu().numpy().flatten()
    sampled_targets = sampled_targets.cpu().numpy().flatten()

    # Create sample indices for the x-axis (1st sample, 2nd sample, etc.)
    sample_indices = list(range(1, num_samples + 1))

    # Plot predictions and ground truth vs sample number
    # plt.figure(figsize=(8, 6))
    # Add vertical lines for each data point
    for x in sample_indices:
        axes[0].axvline(x=x, color='grey', linestyle='-', alpha=0.2,zorder=0)
    axes[0].scatter(sample_indices, sampled_targets, marker='o', label='Ground Truth', fc="none", ec="k",zorder=10)
    axes[0].scatter(sample_indices, np.power(10,sampled_predictions), marker='s', label='Predictions', fc="none",ec="r" ,zorder=10)
    axes[0].set_yscale("log")
    axes[0].set_xlabel('Sample Number')
    axes[0].set_ylabel('Value')

    # Get one batch from the validation loader
    dataiter = iter(test_loader)
    inputs, = next(dataiter)

    # Determine how many samples to take (if batch size < 50, take the whole batch)
    num_samples = min(50, batch_size)

    # Randomly select indices from the batch
    indices = random.sample(range(batch_size), num_samples)
    sampled_inputs = inputs[:,input_mask][indices]
    sampled_targets = inputs[indices,25]

    # Compute model predictions with no gradient computation
    with torch.no_grad():
        sampled_predictions = model(sampled_inputs)

    # Convert tensors to numpy arrays for plotting
    sampled_predictions = sampled_predictions[:,0].cpu().numpy().flatten()
    sampled_targets = sampled_targets.cpu().numpy().flatten()

    # Create sample indices for the x-axis (1st sample, 2nd sample, etc.)
    sample_indices = list(range(1, num_samples + 1))

    # Plot predictions and ground truth vs sample number
    # plt.figure(figsize=(8, 6))
    # Add vertical lines for each data point
    for x in sample_indices:
        axes[1].axvline(x=x, color='grey', linestyle='-', alpha=0.2,zorder=0)
    axes[1].scatter(sample_indices, sampled_targets, marker='o', label='Ground Truth', fc="none", ec="k",zorder=10)
    axes[1].scatter(sample_indices, np.power(10,sampled_predictions), marker='s', label='Predictions', fc="none",ec="r" ,zorder=10)
    axes[1].set_yscale("log")
    axes[1].set_xlabel('Sample Number')
    axes[1].set_ylabel('Value')

    axes[0].set_title(f'Epoch [{epoch}/{num_epochs}] Predictions and Ground Truth vs. Sample Number')
    #plt.legend()
    axes[0].grid()
    axes[1].grid()
    plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0.)
    plt.savefig("samples.png")
    plt.close()


    """
    if time_step < time_step_fiducial:
        weights['time_step'] += 0.1*exp_time_step
        exp_time_step += 1
    else:
        weights['time_step'] += 0
        exp_time_step = 1

    if energy_error > energy_error_fiducial:
        weights['energy_loss'] += 0.1*exp_energy_loss
        exp_energy_loss += 1
    else:
        weights['energy_loss'] += 0
        exp_energy_loss = 1
        """

    #print(f"Epoch [{epoch}/{num_epochs}], Train Loss: {train_loss:.4e}, Test Loss: {test_loss:.4e}, Energy Loss: {energy_error:.4e}/{energy_error_fiducial:.4e}, std: {energy_error_std:.4e}/{energy_error_fiducial_std:.4e}, Time step: {time_step:.4e}/{time_step_fiducial:.4e}")
    #print(f"Epoch [{epoch}/{num_epochs}], Train Loss: {train_res['train_loss']:.4e}, Val Loss: {val_res['val_loss']:.4e}, Energy Loss: {np.log10(train_res['energy_error']):.4e}/{np.log10(train_res['energy_error_fiducial']):.4e}, {np.log10(val_res['energy_error']):.4e}/{np.log10(val_res['energy_error_fiducial']):.4e}, Time step: {train_res['time_step']:.4e}/{train_res['time_step_fiducial']:.4e}, {val_res['time_step']:.4e}/{val_res['time_step_fiducial']:.4e}, {val_res['time_step_std']:.4e}/{val_res['time_step_fiducial_std']:.4e}")
    print(f"Epoch [{epoch}/{num_epochs}], Timestep error: {train_res['time_step_relative_error']:.4e}/{val_res['time_step_relative_error']:.4e}, Energy Loss: {np.log10(train_res['energy_error']):.4e}/{np.log10(train_res['energy_error_fiducial']):.4e}, {np.log10(val_res['energy_error']):.4e}/{np.log10(val_res['energy_error_fiducial']):.4e}, Time step: {train_res['time_step']:.4e}/{train_res['time_step_fiducial']:.4e}, {val_res['time_step']:.4e}/{val_res['time_step_fiducial']:.4e}, {val_res['time_step_std']:.4e}/{val_res['time_step_fiducial_std']:.4e}")
    #print(f"Epoch [{epoch}/{num_epochs}], Train Loss: {train_res['train_loss']:.4e}, Val Loss: {val_res['val_loss']:.4e}, Energy Loss: {np.log10(train_res['energy_error']):.4e}/{np.log10(train_res['energy_error_fiducial']):.4e}, {np.log10(val_res['energy_error']):.4e}/{np.log10(val_res['energy_error_fiducial']):.4e}, energy_pred: {energy_pred:.4e}/{energy_init:.4e}, Time step: {time_step:.4e}/{time_step_fiducial:.4e}")

[2025-04-11 15:57:08,304] [INFO] [logging.py:128:log_dist] [Rank 0] DeepSpeed info: version=0.16.4, git-hash=unknown, git-branch=unknown
[2025-04-11 15:57:08,305] [INFO] [config.py:734:__init__] Config mesh_device None world_size = 1
[2025-04-11 15:57:08,307] [INFO] [logging.py:128:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
[2025-04-11 15:57:08,308] [INFO] [logging.py:128:log_dist] [Rank 0] Using DeepSpeed Optimizer param name adam as basic optimizer
[2025-04-11 15:57:08,308] [INFO] [logging.py:128:log_dist] [Rank 0] Removing param_group that has no 'params' in the basic Optimizer
[2025-04-11 15:57:08,309] [INFO] [logging.py:128:log_dist] [Rank 0] DeepSpeed Basic Optimizer = FusedAdam
[2025-04-11 15:57:08,309] [INFO] [utils.py:59:is_zero_supported_optimizer] Checking ZeRO support for optimizer=FusedAdam type=<class 'deepspeed.ops.adam.fused_adam.FusedAdam'>
[2025-04-11 15:57:08,309] [INFO] [logging.py:128:log_dist] [Rank 0] Creating torch.float32 ZeRO stage 1 optimizer


/tmp/ipykernel_294660/1023649848.py:259: RuntimeWarning: invalid value encountered in log10
  print(f"Epoch [{epoch}/{num_epochs}], Timestep error: {train_res['time_step_relative_error']:.4e}/{val_res['time_step_relative_error']:.4e}, Energy Loss: {np.log10(train_res['energy_error']):.4e}/{np.log10(train_res['energy_error_fiducial']):.4e}, {np.log10(val_res['energy_error']):.4e}/{np.log10(val_res['energy_error_fiducial']):.4e}, Time step: {train_res['time_step']:.4e}/{train_res['time_step_fiducial']:.4e}, {val_res['time_step']:.4e}/{val_res['time_step_fiducial']:.4e}, {val_res['time_step_std']:.4e}/{val_res['time_step_fiducial_std']:.4e}")


Epoch [0/1000], Timestep error: 7.4384e+06/4.8365e+06, Energy Loss: 1.0789e+00/nan, 1.0444e+00/nan, Time step: 2.1187e+00/1.6605e-03, 1.4588e+00/1.7660e-03, 3.5109e-02/2.3513e-03
Epoch [1/1000], Timestep error: 3.4769e+06/5.4943e+05, Energy Loss: 1.0075e+00/nan, 8.8665e-01/nan, Time step: 8.3408e-01/1.6605e-03, 2.2308e-01/1.7660e-03, 7.6768e-02/2.3513e-03
Epoch [2/1000], Timestep error: 2.5004e+05/7.8335e+02, Energy Loss: 5.6112e-01/nan, nan/nan, Time step: 5.5184e-02/1.6605e-03, 2.0388e-03/1.7660e-03, 3.2613e-03/2.3513e-03
Epoch [3/1000], Timestep error: 2.1264e+03/2.3032e+01, Energy Loss: nan/nan, nan/nan, Time step: 6.6371e-04/1.6605e-03, 2.3838e-04/1.7660e-03, 4.6830e-04/2.3513e-03
Epoch [4/1000], Timestep error: 5.5202e+02/1.2012e+01, Energy Loss: nan/nan, nan/nan, Time step: 1.8123e-04/1.6605e-03, 1.5257e-04/1.7660e-03, 3.0615e-04/2.3513e-03
Epoch [5/1000], Timestep error: 3.4472e+02/1.1379e+01, Energy Loss: nan/nan, nan/nan, Time step: 1.3086e-04/1.6605e-03, 1.4248e-04/1.7660e-0

KeyboardInterrupt: 